In [1]:
# ======================
# Imports
# ======================
import math as mt
import scipy.optimize as spo


# ======================
# Scenario Setup
# ======================
# Given data
S1 = 400e3          # Load 1 = 400 kW
pf1 = 0.85           # Power factor (lagging)
S2 = 300e3          # Load 2 = 300 kW
pf2 = 0.75           # Power factor (lagging)
Z_line = 1 + 2.5j    # Line impedance (ohms)
V_source = 13.8e3    # Source RMS voltage (volts)

# Compute power angles
theta1 = mt.acos(pf1)
theta2 = mt.acos(pf2)

# Complex powers for both loads (S = P + jQ)
S_load1 = S1 * (mt.cos(theta1) + 1j * mt.sin(theta1))
S_load2 = S2 * (mt.cos(theta2) + 1j * mt.sin(theta2))


# ======================
# Newton-Raphson Functions
# ======================
def f(V):
    """Function whose root corresponds to the desired bus voltage."""
    S_total = S_load1 + S_load2
    I = S_total.conjugate() / V
    V_send = V + I * Z_line
    return abs(V_send) - V_source


def df_dV(V):
    """Derivative of f(V) with respect to V."""
    # Using a small numerical step (finite difference)
    h = 1e-6
    return (f(V + h) - f(V - h)) / (2 * h)


# ======================
# Newton-Raphson Setup
# ======================
X0 = 12000       # Initial guess (12 kV)
MAX_ITERS = 42   # Iteration limit
X_TOL = 1e-7     # Tolerance

# Run Newton-Raphson method
p_nr, p_nr_info = spo.newton(
    f, X0, fprime=df_dV,
    tol=X_TOL, maxiter=MAX_ITERS,
    full_output=True
)


# ======================
# Results
# ======================
print(f"Finding a root from {{X0}}:")
print(p_nr_info)
print(f"Residual value: {{f(p_nr)}}")
print(f"The RMS voltage at the common bus is {{p_nr / 1000:.4f}} kilovolts.")


Finding a root from {X0}:
      converged: True
           flag: converged
 function_calls: 8
     iterations: 4
           root: 13683.765423596236
         method: newton
Residual value: {f(p_nr)}
The RMS voltage at the common bus is {p_nr / 1000:.4f} kilovolts.
